In [ ]:
import pandas as pd
from Bio import SeqIO
import subprocess
from os import listdir

In [ ]:
references = [
    "AH3Ma.softmasked.fasta",
    "AH3Mb.softmasked.fasta",
    "BCMa.softmasked.fasta",
    "BCMb.softmasked.fasta",
    "BOAXa.softmasked.fasta",
    "BOAXb.softmasked.fasta",
    "EH23a.softmasked.fasta",
    "EH23b.softmasked.fasta",
    "FCS1a.softmasked.fasta",
    "FCS1b.softmasked.fasta",
    "GRMa.softmasked.fasta",
    "GRMb.softmasked.fasta",
    "H3S1a.softmasked.fasta",
    "H3S1b.softmasked.fasta",
    "H3S7a.softmasked.fasta",
    "H3S7b.softmasked.fasta",
    "KOMPa.softmasked.fasta",
    "KOMPb.softmasked.fasta",
    "SAN2a.softmasked.fasta",
    "SAN2b.softmasked.fasta",
    "SODLa.softmasked.fasta",
    "SODLb.softmasked.fasta",
    "WHWa.softmasked.fasta",
    "WHWb.softmasked.fasta"
    ]

In [ ]:
assemblies = [
    "I3.softmasked.fasta",
    "WeddingCakexSSV-133-2.fasta",
    "SundaeDriver.fasta"
]

In [ ]:
def create_directory(dir_name):
    subprocess.run(["mkdir", dir_name])

def split_chrs(reference):
    for seqrd in SeqIO.parse(reference,"fasta"):
        if str(seqrd.id).split('.')[1][:3] == "chr":
            SeqIO.write(seqrd,f"split_chrs/{str(seqrd.id).split('.')[0]}.{str(seqrd.id).split('.')[1]}.fasta","fasta")

def sketch_references(input_list,dir="split_chrs/"):
    input_list = [dir + x for x in input_list]
    cmd = "sourmash"
    args = f"sketch dna -p k=31 --output-dir reference_signatures/".split(" ")
    subprocess.run([cmd] + args + input_list)

def sketch_assembly(assembly):
    cmd = "sourmash"
    args = f"sketch dna -p k=31 --output-dir assembly_signatures/ {assembly}".split(" ")
    subprocess.run([cmd] + args)

def sourmash_compare(assembly):
    cmd = "sourmash"
    args = f"compare --csv compare_results/{assembly}_compare.csv assembly_signatures/{assembly}.sig reference_signatures/".split(" ")
    subprocess.run([cmd] + args)

def process_dataframe(assembly):
    df = pd.read_csv(f"compare_results/{assembly}_compare.csv")
    df.columns = [x.split("/")[-1] for x in df.columns]
    df.index = df.columns
    return df

def get_max_key(df, assembly, filter_str):
    filt_df = df.filter(like=filter_str)
    return max(dict(filt_df.loc[assembly]), key=dict(filt_df.loc[assembly]).get)

def split_references(references):
    create_directory("split_chrs")
    for reference in references:
        split_chrs(reference)

def sourmash(assemblies, references):
    ref_dict = {}
    create_directory("reference_signatures")
    create_directory("assembly_signatures")
    create_directory("compare_results")
    input_list = listdir("split_chrs")
    sketch_references(input_list)
    for assembly in assemblies:
        xy_dict = {}
        ref_list = []
        sketch_assembly(assembly)
        sourmash_compare(assembly)
        df = process_dataframe(assembly)
        for number in [f"chr{i}" for i in range(1,10)]:
            max_key = get_max_key(df, assembly, number)
            ref_list.append(max_key)
        for xy in ["chrX","chrY"]:
            max_key = get_max_key(df, assembly, xy)
            xy_dict[max_key] = dict(df.loc[assembly])[max_key]
        if "chrY" in max(xy_dict, key=xy_dict.get):
            print("Best match to Chromosome Y. Skipping.")
            continue
        else:
            ref_list.append(max(xy_dict, key=xy_dict.get))
            ref_dict[assembly] = ref_list
    return ref_dict

def ragtag(assembly,chr_list,dir="split_chrs/"):
    base = assembly.split(".fasta")[0]
    create_directory(f"ragtag_out/{base}_ragtag")
    with open(f'ragtag_out/{base}_ragtag/{base}_refs.fasta', 'w') as fp: 
        for chr in chr_list:
            cmd = "cat"
            args = f"{dir}{chr}".split(" ")
            subprocess.run([cmd] + args, stdout=fp)
    cmd = "ragtag.py"
    args = f"scaffold -t 8 ragtag_out/{base}_ragtag/{base}_refs.fasta {assembly} -o ragtag_out/{base}_ragtag/{base}_ragtag_tmp/".split(" ")
    subprocess.run([cmd] + args)
    cmd = "mv"
    args = f"ragtag_out/{base}_ragtag/{base}_ragtag_tmp/ragtag.scaffold.fasta ragtag_out/{base}_ragtag/{base}.RagTag.FINAL.fasta".split(" ")
    subprocess.run([cmd] + args)
    cmd = "mv"
    args = f"ragtag_out/{base}_ragtag/{base}_ragtag_tmp/ragtag.scaffold.stats ragtag_out/{base}_ragtag/{base}.RagTag.FINAL.stats".split(" ")
    subprocess.run([cmd] + args)

def main(assemblies, references):
    split_references(references)
    s_out = sourmash(assemblies, references)
    create_directory(f"ragtag_out")
    for assembly in s_out.keys():
        ragtag(assembly,s_out[assembly])

In [ ]:
main(assemblies, references)